In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
from metpy import calc as mpcalc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import datetime


import geocat.viz as gv
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.animation as animation


In [ ]:
HOME = str(Path.home())
credFile = open(HOME+'/.rdarc','r')
userId, pw = credFile.read().split()

In [ ]:
year =  2018
month = 9
day = 1
hour = 0
ntimes = 4


min_lon = -140. + 360.
max_lon =  -5.  + 360.
delta_lon = 20.
min_lat = 0.
max_lat =  30.

# encode the date 
date = datetime.datetime(year,month,day,hour)

In [ ]:
era5_base ='https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/'

In [ ]:
date.strftime('%Y%m')

In [ ]:
varId  = '138'
varNam = 'vo'

In [ ]:
session = requests.Session()
session.auth = (userId, pw)

            

In [ ]:
def animate(i,dateIn,min_lon,max_lon,min_lat,max_lat,Cmin,Cmax):
    central_lon = .5*(min_lon + max_lon)
    delta_lon = 20.
    date = dateIn + pd.Timedelta(days=i*.25)
    
    fnam = date.strftime('%Y%m') + '/e5.oper.an.pl.128_' + varId + '_' + varNam + \
    '.ll025sc.' + date.strftime('%Y%m%d')+'00_' + date.strftime('%Y%m%d')+'23.nc'   
    ds_url = era5_base + fnam
    vorf = xr.backends.PydapDataStore.open(ds_url, session=session)
    vo = ds.VO.sel(time=date,level=850, latitude=slice(max_lat, min_lat), longitude= slice(min_lon, max_lon) )*1.e5
    cf = vo.plot.contourf(ax=ax,
                          transform=projection,
                          levels=np.arange(-20, 20, 2),
                          cmap='bwr',extend='both',
                          add_colorbar=False)

    
    
    plt.title(date, y=1.2)
    

In [ ]:
fnam = date.strftime('%Y%m') + '/e5.oper.an.pl.128_' + varId + '_' + varNam + \
'.ll025sc.' + date.strftime('%Y%m%d')+'00_' + date.strftime('%Y%m%d')+'23.nc'   
ds_url = era5_base + fnam

vorf = xr.backends.PydapDataStore.open(ds_url, session=session)
ds = xr.open_dataset(vorf)
vo = ds.VO.sel(time=date,level=850, latitude=slice(max_lat, min_lat), longitude= slice(min_lon, max_lon) )*1.e5

In [ ]:

# set the min and max values to shade in the IR brightness temperature field
Cmin=  -10
Cmax = 10
Cint = 1

print(vo.min().values, vo.max().values)

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()


# Generate figure and set its size (width, height) in inches
fig = plt.figure(figsize=(10,8))

# Generate axes, using Cartopy, drawing coastlines, and adding features
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.coastlines(linewidths=0.5)
ax.add_feature(cfeature.LAND, facecolor='lightgray')

cf = vo.plot.contourf(ax=ax,
                          transform=projection,
                          levels=np.arange(-20, 20, 2),
                          cmap='bwr',extend='both',
                          add_colorbar=False)
# Add colorbar
cbar = plt.colorbar(cf,
                    shrink=0.8,
                    drawedges=True,
                    ticks=np.arange(-20,20,4),
                    extendrect=True,
                    extendfrac='auto', orientation='horizontal')

lon1tick=280
lon2tick=340

gl = ax.gridlines(draw_labels=True)
#ax.coastlines('50m', linewidth=0.8,color='gray')
gl.top_labels = gl_right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER



#---------------------------------------------------------------------------------------------------
animArgs = (date,min_lon,max_lon,min_lat,max_lat,Cmin,Cmax)
# runs the animation initiated with the frame from init and progressed with the animate function
anim = animation.FuncAnimation(fig,animate,frames=ntimes, interval=200, fargs=animArgs)
anim


